<h3 style="color: #000000; font-weight: bold;"> Gen AI Project: Building a Finantial Bot </h3>

##### **Goal of the Project** : The objective of the project is to deploy a chatbot capable of providing financial advice based on the book **_Rich Dad Poor Dad_** by **Robert T. Kiyosaki**.


In [1]:
# importing the libraries

import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from chromadb.config import Settings
import chromadb
import openai

<h3 style="color: #000000; font-weight: bold;"> ETL </h3>

In [5]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter


document_dir = "./pdfs"
pdf_files = [f for f in os.listdir(document_dir) if f.endswith(".pdf")]
all_chunks = []
total_pages = 0

for filename in pdf_files:
    file_path = os.path.join(document_dir, filename)
    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()
    total_pages += len(pages)
    
    text_splitter = CharacterTextSplitter(chunk_size=10000, chunk_overlap=0)
    chunks = text_splitter.split_documents(pages)
    
    all_chunks.extend(chunks)

print(f"Total chunks from all PDFs: {len(all_chunks)}")
print(f"Total pages from all PDFs: {total_pages}")


Total chunks from all PDFs: 685
Total pages from all PDFs: 685


In [6]:
# checking the content of the PDFs Files/Pages
for filename in pdf_files:
    file_path = os.path.join(document_dir, filename)
    
    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()
    
    print(f"{filename} have {len(pages)} pages.")


O Homem Mais Rico da Babilônia.pdf have 98 pages.
O INVESTIDOR INTELIGENTE - BENJAMIN GRAHAM.pdf have 587 pages.


# PDF location for retrieving the data
document_dir = "./"
filename = "Rich Dad Poor Dad.pdf"
file_path = os.path.join(document_dir, filename)

In [4]:
# terminal: echo OPENAI_API_KEY="" > .env

load_dotenv()

True

In [5]:
load_dotenv()
# api_key = os.getenv("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
api_key =""

In [6]:
client = chromadb.PersistentClient(path="./chroma_db") # initializes the new environment , what kind of client

In [7]:
db = Chroma.from_documents(chunks, embeddings, persist_directory="./chroma_db", client=client)

<h3 style="color: #000000; font-weight: bold;"> System Architecture </h3>

In [8]:
user_question = "O que devo fazer se eu quero começar investir?" # User question stored in a variable
retrieved_docs = db.similarity_search(user_question, k=5) # k is the number of documents to retrieve the context

In [9]:
# Display top results
for i, doc in enumerate(retrieved_docs[:1]): # Display top 3 results
    print(f"Document {i+1}:\n{doc.page_content[36:1000]}") # Display content 

Document 1:
timento mensal de US$50, mas se esse é
todo o dinheiro que você dispõe para gastar, então essas páginas de investimento
micro são a única maneira de construir uma carteira diversificada.
Você também pode comprar ações individuais diretamente das companhias
emitentes. Em 1994, o Securities and Exchange Commission afrouxou as
algemas há muito colocadas na venda direta de ações ao público. Centenas de
companhias responderam a essa mudança com a criação de programas na
internet que permitem aos investidores comprar ações sem passar por um
corretor. Algumas fontes de informação on-line grau de investimento sobre a
compra direta de ações incluem a www.dripcentral.com, www.netstockdirect.
com (uma afiliada da Sharebuilder) e a www.stockpower.com. Com freqüência,
é necessário pagar diversas taxas incômodas que podem exceder US$25 ao ano.
Mesmo assim, os programas de compra direta de ações são, em geral, mais
baratos do que os corretores.
Fique avisado, no en


<h3 style="color: #000000; font-weight: bold;"> Deployment & System Integration </h3>

In [10]:
def _get_document_prompt(docs):
    prompt = "\n"
    for doc in docs:
        prompt += "\nContent:\n"
        prompt += doc.page_content + "\n\n"
    return prompt

In [11]:
# Generate a formatted context from the retrieved documents
formatted_context = _get_document_prompt(retrieved_docs)
print("Context formatted for GPT model.{}".format(formatted_context))

Context formatted for GPT model.

Content:
mordida monstruosa de 8% em um investimento mensal de US$50, mas se esse é
todo o dinheiro que você dispõe para gastar, então essas páginas de investimento
micro são a única maneira de construir uma carteira diversificada.
Você também pode comprar ações individuais diretamente das companhias
emitentes. Em 1994, o Securities and Exchange Commission afrouxou as
algemas há muito colocadas na venda direta de ações ao público. Centenas de
companhias responderam a essa mudança com a criação de programas na
internet que permitem aos investidores comprar ações sem passar por um
corretor. Algumas fontes de informação on-line grau de investimento sobre a
compra direta de ações incluem a www.dripcentral.com, www.netstockdirect.
com (uma afiliada da Sharebuilder) e a www.stockpower.com. Com freqüência,
é necessário pagar diversas taxas incômodas que podem exceder US$25 ao ano.
Mesmo assim, os programas de compra direta de ações são, em geral, mais
baratos

<h3 style="color: #000000; font-weight: bold;"> NLP & Promp engineering </h3>

In [12]:
prompt = f"""
## SYSTEM ROLE
You are a highly knowledgeable and creative chatbot designed to assist with financial advice based on the principles outlined in a selection of financial books. Your responses must be rooted exclusively in the provided content, focusing on explaining, comparing, and contextualizing key concepts from the books **The Richest Man in Babylon**, **The Intelligent Investor**, and **Secrets of the Millionaire Mind**.

## USER QUESTION
The user has asked:  
**"{user_question}"**

## CONTEXT
Here is the relevant content from the provided books:  
'''
{formatted_context}
'''

## GUIDELINES
1. **Accuracy**:  
   - Use only the information provided in the `CONTEXT` section to answer the question.  
   - If the context does not contain relevant information, clearly state: "The provided context does not contain this information."  
   - If more than one book is relevant, draw on all of them to give a comprehensive and nuanced answer. 
   - Begin by explaining the significance of the user's question and what it implies in the context of the books.  
   - Use analogies where needed to make complex ideas more digestible.

2. **Formatting**:  
   - Keep the response concise.  
   - If the answer is too long, split it into shorter sentences and continue on the next line.  
   - Ensure the user's question appears in **bold** to differentiate it from the answer.
   - If content from multiple books is relevant, combine the information in a coherent, integrated way. 

3. **Transparency**:  
   - Reference the book title whenever possible to provide credibility to the response.  
   - Clearly reference the page(s) or chapters that correspond to the provided content.

4. **Clarity**:  
   - Use concise, professional, and user-friendly language.  
   - Format the response in **Markdown** for enhanced readability.

## TASK
1. Provide a direct and comprehensive answer to the user's question, synthesizing information from all three books if they complement each other.
2. Use an analogy to simplify complex concepts and make the response more relatable.  
3. Deliver the response in the following format:

## RESPONSE FORMAT
'''
# [Title Reflecting the Answer]
**User Question:**  
**[The user's question.]**

**Answer:**  
[Provide a clear and concise answer to the question, integrating insights from **The Richest Man in Babylon**, **The Intelligent Investor**, and **Secrets of the Millionaire Mind**. If the context from one book is not enough, refer to the other books to provide a more rounded response.]

**Analogy:**  
[Provide a simple and engaging analogy to support the explanation.]

**Source:**  
• *The Richest Man in Babylon*, Page(s): [Insert relevant page range]  
• *The Intelligent Investor*, Page(s): [Insert relevant page range]  
• *Secrets of the Millionaire Mind*, Page(s): [Insert relevant page range]
'''
"""
print("Prompt constructed.")


Prompt constructed.


<h3 style="color: #000000; font-weight: bold;"> Maintenance & Improvements </h3>

In [13]:
# Set up GPT client and parameters
client = openai.OpenAI()
model_params = {
    'model': 'gpt-4o',
    'temperature': 0.7,  # Increase creativity
    'max_tokens': 4000,  # Allow for longer responses
    'top_p': 0.9,        # Use nucleus sampling
    'frequency_penalty': 0.5,  # Reduce repetition
    'presence_penalty': 0.6    # Encourage new topics
}

In [14]:
messages = [{'role': 'user', 'content': prompt}]
completion = client.chat.completions.create(messages=messages, **model_params, timeout=120)

In [15]:
answer = completion.choices[0].message.content
print(answer)

'''
# Iniciando no Mundo dos Investimentos

**User Question:**  
**"O que devo fazer se eu quero começar investir?"**

**Answer:**  
Para começar a investir, é essencial seguir algumas diretrizes fundamentais. Primeiramente, de acordo com *The Intelligent Investor*, é importante diversificar seus investimentos. Benjamin Graham recomenda possuir entre dez a trinta ações diferentes para evitar exposição excessiva a um único setor (Páginas 6-7). Além disso, você pode considerar o uso de corretoras online especializadas como Sharebuilder e FolioFN, que permitem compras automáticas e não exigem saldos mínimos elevados (Páginas 5-6).

É igualmente importante manter-se disciplinado e evitar a tentação de especular ou negociar excessivamente. A internet facilita as atualizações instantâneas do mercado, mas o investidor defensivo deve focar em uma estratégia de longo prazo e estável (Páginas 6-7). 

Por fim, caso prefira orientação profissional, considere usar uma corretora com serviços complet